## BERT (Bidirectional Encoder Representation from Transformers)

<img src="Screenshot 2025-05-09 at 01.27.33.png" width="500" />

- Apa itu Attention (udah kamu kuasai feeling-nya 🔥)

- Bagaimana Attention dihitung (Query-Key-Value + hitungan kecil → lagi kita lakuin sekarang)

- Self-Attention di semua kata di kalimat (next step kita)

- Multi-Head Attention (kenapa pakai banyak Attention barengan?)

- Positional Encoding (gimana BERT tau urutan kata?) -> Learned Positional Embedding.

- Encoder Stack (BERT = tumpukan encoder, kayak kue lapis)

- Training Objective BERT (Masked Language Model + Next Sentence Prediction)

- Fine-Tuning (gimana BERT dipakai buat tugas kayak QA, Sentimen, dst.)

# SCALED DOT PRODUCT ATTENTION

# FULL STACK ENCODER

# PRETRAINING 

## MLM (MASKED LANGUAGE MODELLING)


## NSP (NEXT SENTENCE PREDICTION)

# FINE-TUNING

## Text Classification

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer
)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pretrained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)  # Move model to appropriate device

# Load dataset
dataset = load_dataset("imdb", split="train[:2000]")
dataset = dataset.train_test_split(test_size=0.2)
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

# Tokenize function - create a function that properly formats data
def tokenize_function(examples):
    # Return tokenized examples with proper padding and truncation
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,  # Specify max_length explicitly
        return_tensors="pt"  # Return PyTorch tensors
    )

# Process dataset to have the right format and data types
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]  # Remove text column which won't be needed
)

# Convert label column to make it compatible with the model
tokenized_datasets = tokenized_datasets.map(
    lambda examples: {"labels": examples["label"]},
    remove_columns=["label"]  # Remove original label column
)

# Check the structure of our processed dataset
print("\nSample from processed dataset:")
sample = tokenized_datasets["train"][0]
print(f"Type: {type(sample)}")
print(f"Keys: {sample.keys()}")
print(f"Example item: {sample}")

# Define training arguments
# Check the TrainingArguments available parameters
from inspect import signature
print("Available parameters for TrainingArguments:", signature(TrainingArguments))

training_args = TrainingArguments(
    output_dir="./results",
    # Try with no evaluation strategy parameter first
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    # Simplified arguments to minimize potential issues
    logging_dir="./logs"
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Train model
print("\nStarting training...")
trainer.train()

# Evaluate model
print("\nEvaluating model...")
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Save model
model_path = "./imdb-bert-classifier"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"\nModel saved to {model_path}")

## SQuAD 

In [ ]:
from transformers import BertTokenizerFast, BertForQuestionAnswering
from transformers import Trainer, TrainingArguments
from datasets import load_dataset


model_name = "bert-base-uncased"

tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)


dataset = load_dataset("squad", split="train[:1000]")
dataset = dataset.train_test_split(test_size=0.2)


def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = [c.strip() for c in examples["context"]]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="pt",
    )

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(inputs["offset_mapping"]):
        answer = examples["answers"][i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        sequence_ids = inputs.sequence_ids(i)

        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - list(reversed(sequence_ids)).index(1)

        offsets = offsets[context_start:context_end+1]
        for idx, (start, end) in enumerate(offsets):
            if start <= start_char and end >= start_char:
                start_positions.append(idx + context_start)
            if start <= end_char and end >= end_char:
                end_positions.append(idx + context_start)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)


training_args = TrainingArguments(
    output_dir="./qa-results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)


trainer.train()

# Output

## HIDDEN STATE (HIDDEN OUTPUT)

## POOLER OUTPUT ([CLS])

| Output        | Bentuk                                       | Gunanya                                                                                  |
| :------------ | :------------------------------------------- | :--------------------------------------------------------------------------------------- |
| Hidden States | `(batch_size, sequence_length, hidden_size)` | Tiap token punya representasi kontekstual. Cocok buat tagging per token (NER, QA).       |
| Pooler Output | `(batch_size, hidden_size)`                  | Representasi seluruh kalimat. Cocok buat klasifikasi kalimat (Pos/Neg, Entailment, dst). |


# HYPERPARAMETER

| Hyperparameter | BERT Umum      | Kenapa              |
| -------------- | -------------- | ------------------- |
| Learning Rate  | 2e-5 atau 3e-5 | Supaya update halus |
| Batch Size     | 8 atau 16      | Supaya hemat memori |
| Epoch          | 2-4            | Cukup buat adaptasi |


# QA MANUALLY

- Masukin input ➔ dapat logits (start_logits, end_logits).

- Cari index paling tinggi dari start_logits ➔ itu awal jawaban.

- Cari index paling tinggi dari end_logits ➔ itu akhir jawaban.

- Potong token dari start sampai end ➔ decode jadi teks jawaban.



In [2]:
from transformers import BertTokenizerFast, BertForQuestionAnswering
import torch

# Load model
model_name = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

# Input
context = "The capital city of France is Paris."
question = "What is the capital of France?"

# Tokenize
inputs = tokenizer(question, context, return_tensors="pt")

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits

# Get the highest scoring start and end token positions
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)

# Decode the answer
answer_ids = inputs["input_ids"][0][start_index : end_index + 1]
answer = tokenizer.decode(answer_ids)

print(f"Predicted Answer: {answer}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted Answer: 


# MLM (MASKED LANGUAGE MODELLING)

In [3]:
from transformers import BertTokenizerFast, BertForMaskedLM
import torch

# Load model
model_name = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# Text with mask
text = "The capital of [MASK] is Paris."

# Tokenize
inputs = tokenizer(text, return_tensors="pt")

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits

# Index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

# Predicted token
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
predicted_token = tokenizer.decode(predicted_token_id)

print(f"Predicted Token: {predicted_token}")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Predicted Token: france


### EMBEDDING LAYER

(2, 3, 10)


### SCALED DOT PRODUCT ATTENTION

(1, 3, 1, 1, 10)


### MULTIHEAD ATTENTION

input : [[[-0.77281009 -0.26525163  0.49480591 -0.71234687  1.55256265
    0.03180343  0.97385523 -1.85170586]
  [-0.57396265 -0.72822389 -1.92776488  1.28525226  0.62958119
    0.74690941  0.74743178  0.4630312 ]
  [ 0.27997036  0.23979904 -0.09229853 -0.70767435 -0.09800739
   -0.34597603  1.06333414  0.25265971]
  [-1.82387414  0.24315024 -0.23913714  0.07299795 -1.43867199
    0.2218916   0.79692588 -2.53512674]]

 [[-0.99488304  0.85132085  0.08817293  1.058072    0.68034666
   -0.88408008 -0.6120395  -0.78602775]
  [ 0.68272199  0.24199436  0.38193769 -0.50565454 -0.26785863
    1.48656754 -1.38883187 -0.58510711]
  [-0.64977469  0.68919681  1.23141812  0.39152976 -1.00440959
   -0.97901076 -1.48389504 -0.96477045]
  [ 0.32549657  1.18308707 -0.0607689   1.34716348  0.15218075
    0.48560845  0.78811011 -0.05491841]]]

shape of input : (2, 4, 8)

batch_size = sentences : 2
seq_len = words : 4
d_model = dimention : 8

output : [[[-0.00044888 -0.00099372 -0.00042065 -0.00070045 -0.

### FFN


Input shape: (2, 3, 4)
Output shape: (2, 3, 4)
Output:
 [[[24.93228925 25.01672739 28.52430536 27.89781689]
  [25.15489125 24.22581311 27.73751544 26.67556067]
  [36.99291892 36.90875791 41.84860775 40.34837315]]

 [[26.06460181 25.73991014 29.46731452 28.66470778]
  [35.34297198 34.76383522 39.95592629 39.16061508]
  [23.08314383 22.70280699 25.94482227 25.20208626]]]


### STACK ENCODER - 3 LAYER

Output shape: (2, 6, 768)


## NEXT SENTENCE PREDICTION (NSP)

- Dikasih dua kalimat, suruh BERT tebak:

- Nyambung? (A ➔ B)

- Atau ngaco? (A ➔ random)

- Kalimat 1: "Saya pergi ke pasar."
- Kalimat 2: "Saya membeli buah."
- ==> Label: IsNext (nyambung)

- Kalimat 1: "Saya pergi ke pasar."
- Kalimat 2: "Bulan purnama sangat indah."
- ==> Label: NotNext (acak)
